# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-11-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-11-2020'

In [107]:
# Check out the first 5 rows to make sure the data loaded correctly
jh_live_global.head(20)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,88888.0,NaN,Diamond Princess,US,2020-08-04 02:27:56,NaN,NaN,49,0,0,49.0,"Diamond Princess, US",NaN,0.000000
1,99999.0,NaN,Grand Princess,US,2020-08-04 02:27:56,NaN,NaN,103,3,0,100.0,"Grand Princess, US",NaN,2.912621
2,NaN,NaN,NaN,Antigua and Barbuda,2020-08-12 04:27:29,17.060800,-61.796400,92,3,76,13.0,Antigua and Barbuda,93.946573,3.260870
3,80023.0,Out of ME,Maine,US,2020-08-07 22:34:20,NaN,NaN,0,0,0,0.0,"Out of ME, Maine, US",NaN,NaN
4,NaN,NaN,NaN,Eritrea,2020-08-12 04:27:29,15.179400,39.782300,285,0,248,37.0,Eritrea,8.036257,0.000000
5,31153.0,Sarpy,Nebraska,US,2020-08-12 04:27:29,41.113142,-96.111564,2323,11,0,2312.0,"Sarpy, Nebraska, US",1240.945319,0.473526
6,31121.0,Merrick,Nebraska,US,2020-08-12 04:27:29,41.166243,-98.037489,63,0,0,63.0,"Merrick, Nebraska, US",812.379110,0.000000
7,31123.0,Morrill,Nebraska,US,2020-08-12 04:27:29,41.717118,-103.006141,59,0,0,59.0,"Morrill, Nebraska, US",1271.003878,0.000000
8,31033.0,Cheyenne,Nebraska,US,2020-08-12 04:27:29,41.219994,-102.994169,26,0,0,26.0,"Cheyenne, Nebraska, US",291.806958,0.000000
9,31045.0,Dawes,Nebraska,US,2020-08-12 04:27:29,42.719850,-103.131377,10,0,0,10.0,"Dawes, Nebraska, US",116.427989,0.000000


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,36710,36747,36782,36829,36896,37015,37054,37054,37162,37269
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,5519,5620,5750,5889,6016,6151,6275,6411,6536,6676
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,31465,31972,32504,33055,33626,34155,34693,35160,35712,36204


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1284,1288,1288,1294,1298,1307,1312,1312,1328,1344
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,166,172,176,182,188,189,193,199,200,205
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1231,1239,1248,1261,1273,1282,1293,1302,1312,1322


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,25510,25669,25669,25742,25840,25903,25960,25960,26228,26415
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3018,3031,3031,3123,3155,3227,3268,3342,3379,3480
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,21419,21901,22375,22802,23238,23667,24083,24506,24920,25263


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,368,375,389,397,411,411,411,418,418,449
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,45,46,46,46,47,47,48,48,49,49


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,5,5,5,5,5,5,5,5,5,5
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,2,2,2,2,2,2,2,2,2,2


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
832,1023.0,Choctaw,Alabama,US,2020-08-12 04:27:29,32.022273,-88.265644,289,12,0,277.0,"Choctaw, Alabama, US",2295.654937,4.152249
1323,1095.0,Marshall,Alabama,US,2020-08-12 04:27:29,34.369760,-86.304867,3194,38,0,3156.0,"Marshall, Alabama, US",3300.473268,1.189731
1322,1085.0,Lowndes,Alabama,US,2020-08-12 04:27:29,32.159728,-86.651584,571,24,0,547.0,"Lowndes, Alabama, US",5870.861608,4.203152


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,5141208,164537,1714960
Brazil,3057470,101752,2449338
India,2329638,46091,1639599


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,5141208,164537,1714960,3261711,2020-08-12 04:27:29,37.909085,-91.336884
Brazil,3057470,101752,2449338,506380,2020-08-12 04:27:29,-12.669522,-48.480493
India,2329638,46091,1639599,643948,2020-08-12 04:27:29,23.088275,81.806127
Russia,895691,15103,701796,178792,2020-08-12 04:27:29,54.546312,62.120860
South Africa,566109,10751,426125,129233,2020-08-12 04:27:29,-30.559500,22.937500


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,578946,10523,0
Florida,542792,8553,0
Texas,517700,9222,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,578946,10523,0,568423,2020-08-12 04:27:29,37.843962,-120.728594
Florida,542792,8553,0,534239,2020-08-12 04:27:29,28.940755,-82.700744
Texas,517700,9222,0,508478,2020-08-12 04:27:29,31.660643,-98.653069
New York,422003,32787,0,389216,2020-08-12 04:27:29,42.671593,-75.579694
Georgia,222588,4351,0,218237,2020-08-12 04:27:29,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
New York,New York City,228343,23592,0
California,Los Angeles,210874,4999,0
Florida,Miami-Dade,135130,1909,0
Arizona,Maricopa,126791,2382,0
Illinois,Cook,112484,4933,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
New York,New York City,228343,23592,0,204751,2020-08-12 04:27:29,40.767273,-73.971526,36061.0
California,Los Angeles,210874,4999,0,205875,2020-08-12 04:27:29,34.308284,-118.228241,6037.0
Florida,Miami-Dade,135130,1909,0,133221,2020-08-12 04:27:29,25.611236,-80.551706,12086.0
Arizona,Maricopa,126791,2382,0,124409,2020-08-12 04:27:29,33.348359,-112.491815,4013.0
Illinois,Cook,112484,4933,0,107551,2020-08-12 04:27:29,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-09,37054,6411,35160,955,1672,92,246499,40410,21397,22033,...,312574,1353,30609,25805,841,14208,10,1804,8085,4649
2020-08-10,37162,6536,35712,963,1679,92,253868,40433,21713,22106,...,313392,1364,31304,26800,847,14510,10,1832,8210,4748
2020-08-11,37269,6676,36204,963,1735,92,260911,40593,22127,22245,...,313394,1385,31747,27938,866,14875,10,1831,8275,4818


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-09,1312,199,1302,52,75,3,4606,791,313,721,...,46659,37,194,223,13,97,1,515,235,104
2020-08-10,1328,200,1312,52,78,3,4764,796,331,723,...,46611,37,200,229,15,100,1,518,241,104
2020-08-11,1344,205,1322,52,80,3,5004,803,352,723,...,46611,37,204,238,16,104,1,523,241,104


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-09,25960,3342,24506,839,567,76,108242,32520,11876,19923,...,1451,1125,22042,13356,395,7945,8,913,6698,1437
2020-08-10,26228,3379,24920,839,569,76,108242,32616,12144,20010,...,1463,1146,22992,16930,399,8045,8,915,6802,1524
2020-08-11,26415,3480,25263,839,575,76,181389,33157,12395,20123,...,1464,1157,23704,19706,451,8181,8,915,7004,1524


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20
130,84001097,US,USA,840,1097.0,Mobile,Alabama,US,30.784723,-88.208424,...,8998,9086,9170,9269,9420,9565,9836,9947,10517,10581
109,84001055,US,USA,840,1055.0,Etowah,Alabama,US,34.045673,-86.040519,...,1914,1930,1957,1967,2041,2075,2104,2121,2162,2180
110,84001057,US,USA,840,1057.0,Fayette,Alabama,US,33.720769,-87.738866,...,169,174,176,178,183,198,204,207,212,223
111,84001059,US,USA,840,1059.0,Franklin,Alabama,US,34.442353,-87.842895,...,1184,1196,1206,1215,1238,1258,1270,1274,1288,1297
112,84001061,US,USA,840,1061.0,Geneva,Alabama,US,31.093890,-85.835728,...,226,232,242,243,248,255,257,261,263,265


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-09,101334,3711,0,186923,49383,559746,50642,50320,15575,49,...,122712,503328,44127,1459,528,100086,63072,7695,60554,3050
2020-08-10,103020,3774,0,187523,50028,574231,51020,50567,15634,49,...,123914,509581,44390,1462,547,100749,63647,7757,61061,3042
2020-08-11,103851,3819,0,188737,50411,578946,51422,50684,15699,49,...,124915,517700,44752,1472,576,101745,64151,7879,61785,3073


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-09        1169    3544     575  438    800     464    764    1784   
2020-08-10        1174    3651     576  442    812     466    767    1807   
2020-08-11        1162    3670     579  444    816     481    770    1819   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-09          841      273  ...     134      5       74       40   
2020-08-10          844      275  ...     135      5       73       39   
2020-08-11          847      276  ...     135      5       73       39   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-09            259   374   278          0       77      5  
2020-08-10            260   370   277          0       77      6  
2020-08-11            267   372   279          0       79      5  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-09,1768,26,0,4150,544,10377,1858,4441,591,0,...,1223,9157,336,58,9,2326,1688,139,998,28
2020-08-10,1797,26,0,4154,555,10476,1862,4444,591,0,...,1233,9230,345,58,9,2327,1697,141,998,28
2020-08-11,1847,26,0,4199,566,10523,1874,4444,591,0,...,1271,9222,349,58,9,2344,1716,147,1006,29


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-09          21      25       5    5      4      11     36      14   
2020-08-10          21      25       5    5      4      11     36      17   
2020-08-11          22      29       6    5      5      11     36      18   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-09           38        7  ...       0      0        0        0   
2020-08-10           38        8  ...       0      0        0        0   
2020-08-11           38        8  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-09              0     0     0         27        0      0  
2020-08-10              0     0     0         27        0      0  
2020-08-11              0     0     0         28        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-09,0.000000,0.021673,0.013461,0.000000,0.063613,0.0,0.019387,0.005599,0.014845,0.005201,...,0.003573,0.013483,0.032274,0.033813,0.035714,0.020103,0.0,0.003895,0.023029,0.016175
2020-08-10,0.002915,0.019498,0.015700,0.008377,0.004187,0.0,0.029895,0.000569,0.014768,0.003313,...,0.002617,0.008130,0.022706,0.038558,0.007134,0.021256,0.0,0.015521,0.015461,0.021295
2020-08-11,0.002879,0.021420,0.013777,0.000000,0.033353,0.0,0.027743,0.003957,0.019067,0.006288,...,0.000006,0.015396,0.014152,0.042463,0.022432,0.025155,0.0,-0.000546,0.007917,0.014743


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-09,0.000000,0.031088,0.006961,0.0,0.071429,0.0,0.018351,0.007643,0.061017,0.000000,...,0.000171,0.0,0.037433,0.037209,0.300000,0.010417,0.0,0.005859,0.157635,0.019608
2020-08-10,0.012195,0.005025,0.007680,0.0,0.040000,0.0,0.034303,0.006321,0.057508,0.002774,...,-0.001029,0.0,0.030928,0.026906,0.153846,0.030928,0.0,0.005825,0.025532,0.000000
2020-08-11,0.012048,0.025000,0.007622,0.0,0.025641,0.0,0.050378,0.008794,0.063444,0.000000,...,0.000000,0.0,0.020000,0.039301,0.066667,0.040000,0.0,0.009653,0.000000,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-09,0.000000,0.022644,0.017564,0.0,0.005319,0.0,0.000000,0.003859,0.000168,0.005603,...,0.001380,0.011691,0.049319,0.000000,0.000000,0.031015,0.0,0.003297,0.041518,0.014831
2020-08-10,0.010324,0.011071,0.016894,0.0,0.003527,0.0,0.000000,0.002952,0.022567,0.004367,...,0.008270,0.018667,0.043100,0.267595,0.010127,0.012587,0.0,0.002191,0.015527,0.060543
2020-08-11,0.007130,0.029891,0.013764,0.0,0.010545,0.0,0.675773,0.016587,0.020669,0.005647,...,0.000684,0.009599,0.030967,0.163969,0.130326,0.016905,0.0,0.000000,0.029697,0.000000


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-09,0.011590,0.027124,NaN,0.004385,0.011719,0.009665,0.006679,0.000000,0.004709,0.0,...,0.017639,0.011446,0.008594,0.003439,0.000000,0.009043,0.008781,0.017453,0.010362,0.011609
2020-08-10,0.016638,0.016977,NaN,0.003210,0.013061,0.025878,0.007464,0.004909,0.003788,0.0,...,0.009795,0.012423,0.005960,0.002056,0.035985,0.006624,0.009117,0.008057,0.008373,-0.002623
2020-08-11,0.008066,0.011924,NaN,0.006474,0.007656,0.008211,0.007879,0.002314,0.004158,0.0,...,0.008078,0.015933,0.008155,0.006840,0.053016,0.009886,0.007919,0.015728,0.011857,0.010191


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-09      0.076427  0.011993  0.003490  0.028169  0.015228  0.002160   
2020-08-10      0.004277  0.030192  0.001739  0.009132  0.015000  0.004310   
2020-08-11     -0.010221  0.005204  0.005208  0.004525  0.004926  0.032189   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-08-09      0.007916  0.013061  0.001190  0.011111  ...  0.015152    0.0   
2020-08-10      0.003927  0.012892  0.003567  0.007326  ...  0.007463    0.0   
2020-08-11      0.003911  0.006641  0.003555  0.003636  ...  0.000000    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-09      0.027778  0.052632   0.007782  0.010811  0.014599        NaN   
2020-08-10     -0.013514 -0.025000   0.003861 -0.010695 -0.003597        NaN   
2020-08-11      0.000000  0.000000   0.026923  0.005405  0.007220        NaN   

Province_State                      
Admin2          Washakie    Weston  
2020-08-09      0.100000  0.000000  
2020-08-10      0.000000  0.200000  
2020-08-11      0.025974 -0.166667  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-09,0.007407,0.0,NaN,0.003142,0.016822,0.006792,0.000539,0.000000,0.001695,NaN,...,0.006584,0.010930,0.002985,0.0,0.0,0.001723,0.000000,0.061069,0.002008,0.000000
2020-08-10,0.016403,0.0,NaN,0.000964,0.020221,0.009540,0.002153,0.000676,0.000000,NaN,...,0.008177,0.007972,0.026786,0.0,0.0,0.000430,0.005332,0.014388,0.000000,0.000000
2020-08-11,0.027824,0.0,NaN,0.010833,0.019820,0.004486,0.006445,0.000000,0.000000,NaN,...,0.030819,-0.000867,0.011594,0.0,0.0,0.007306,0.011196,0.042553,0.008016,0.035714


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                               \
Admin2           Autauga   Baldwin Barbour Bibb Blount Bullock Butler   
2020-08-09      0.000000  0.041667     0.0  0.0   0.00     0.0    0.0   
2020-08-10      0.000000  0.000000     0.0  0.0   0.00     0.0    0.0   
2020-08-11      0.047619  0.160000     0.2  0.0   0.25     0.0    0.0   

Province_State                               ... Wyoming                  \
Admin2           Calhoun Chambers  Cherokee  ...    Park Platte Sheridan   
2020-08-09      0.076923      0.0  0.000000  ...     NaN    NaN      NaN   
2020-08-10      0.214286      0.0  0.142857  ...     NaN    NaN      NaN   
2020-08-11      0.058824      0.0  0.000000  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-09          NaN        NaN   NaN   NaN   0.000000      NaN    NaN  
2020-08-10          NaN        NaN   NaN   NaN   0.000000      NaN    NaN  
2020-08-11          NaN        NaN   NaN   NaN   0.037037      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-09,0.000851,0.021457,0.014770,0.001933,0.048628,0.000104,0.024397,0.005308,0.017618,0.004990,...,0.003131,0.010149,0.027594,0.034941,0.037961,0.019730,1.973730e-16,0.004551,0.034459,0.021553
2020-08-10,0.001883,0.020478,0.015235,0.005155,0.026407,0.000052,0.027146,0.002939,0.016193,0.004152,...,0.002874,0.009140,0.025150,0.036750,0.022548,0.020493,9.868649e-17,0.010036,0.024960,0.021424
2020-08-11,0.002381,0.020949,0.014506,0.002578,0.029880,0.000026,0.027444,0.003448,0.017630,0.005220,...,0.001440,0.012268,0.019651,0.039606,0.022490,0.022824,4.934325e-17,0.004745,0.016439,0.018083


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-09,0.002208,0.025486,0.007659,5.442270e-19,0.057293,6.065551e-36,0.024621,0.007731,0.056928,0.000545,...,0.000817,0.000547,0.035261,0.035423,0.168621,0.016911,0.0,0.004332,0.092467,0.040757
2020-08-10,0.007202,0.015256,0.007670,2.721135e-19,0.048647,3.032775e-36,0.029462,0.007026,0.057218,0.001660,...,-0.000106,0.000274,0.033094,0.031164,0.161226,0.023919,0.0,0.005078,0.058999,0.020379
2020-08-11,0.009625,0.020128,0.007646,1.360567e-19,0.037144,1.516388e-36,0.039920,0.007910,0.060331,0.000830,...,-0.000053,0.000137,0.026547,0.035233,0.113923,0.031960,0.0,0.007366,0.029500,0.010189


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-09,0.001235,0.019095,0.017957,0.002102,0.021088,0.001362,0.020023,0.009064,0.013759,0.005881,...,0.001118,0.013455,0.046378,0.020877,0.003204,0.045124,1.156482e-18,0.005158,0.044903,0.033195
2020-08-10,0.005780,0.015083,0.017426,0.001051,0.012308,0.000681,0.010011,0.006008,0.018163,0.005124,...,0.004694,0.016061,0.044739,0.144236,0.006665,0.028855,5.782412e-19,0.003674,0.030215,0.046869
2020-08-11,0.006455,0.022487,0.015595,0.000526,0.011426,0.000341,0.342892,0.011298,0.019416,0.005385,...,0.002689,0.012830,0.037853,0.154103,0.068496,0.022880,2.891206e-19,0.001837,0.029956,0.023435


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-09,0.014815,0.022776,NaN,0.005591,0.014724,0.011527,0.007870,0.000345,0.004722,3.816302e-45,...,0.017391,0.013066,0.009288,0.003594,0.013253,0.011678,0.011170,0.017392,0.014093,0.010537
2020-08-10,0.015726,0.019877,NaN,0.004400,0.013893,0.018702,0.007667,0.002627,0.004255,1.908151e-45,...,0.013593,0.012744,0.007624,0.002825,0.024619,0.009151,0.010143,0.012725,0.011233,0.003957
2020-08-11,0.011896,0.015900,NaN,0.005437,0.010774,0.013457,0.007773,0.002470,0.004206,9.540756e-46,...,0.010836,0.014339,0.007889,0.004833,0.038818,0.009519,0.009031,0.014226,0.011545,0.007074


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-09      0.046483  0.014793  0.004664  0.026598  0.016441  0.008379   
2020-08-10      0.025380  0.022493  0.003201  0.017865  0.015720  0.006345   
2020-08-11      0.007579  0.013848  0.004205  0.011195  0.010323  0.019267   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-08-09      0.004472  0.020597  0.002719  0.018023  ...  0.015462   
2020-08-10      0.004199  0.016744  0.003143  0.012674  ...  0.011462   
2020-08-11      0.004055  0.011693  0.003349  0.008155  ...  0.005731   

Province_State                                                         \
Admin2                Platte  Sheridan  Sublette Sweetwater     Teton   
2020-08-09      4.768377e-07  0.032131  0.025870   0.006526  0.007325   
2020-08-10      2.384188e-07  0.009309  0.000435   0.005193 -0.001685   
2020-08-11      1.192094e-07  0.004654  0.000218   0.016058  0.001860   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2020-08-09      0.009598       -1.0  0.079890  0.000015  
2020-08-10      0.003000       -1.0  0.039945  0.100008  
2020-08-11      0.005110       -1.0  0.032960 -0.033330  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-09,0.009820,0.010442,NaN,0.009689,0.019390,0.011369,0.000734,0.000123,0.001972,NaN,...,0.010888,0.019576,0.005055,0.001105,0.007848,0.003733,0.005299,0.042453,0.005122,0.004935
2020-08-10,0.013112,0.005221,NaN,0.005326,0.019805,0.010454,0.001444,0.000399,0.000986,NaN,...,0.009532,0.013774,0.015920,0.000553,0.003924,0.002081,0.005315,0.028421,0.002561,0.002468
2020-08-11,0.020468,0.002611,NaN,0.008080,0.019813,0.007470,0.003944,0.000200,0.000493,NaN,...,0.020176,0.006454,0.013757,0.000276,0.001962,0.004693,0.008256,0.035487,0.005288,0.019091


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-08-09      0.001563  0.032337  0.000122  0.074870  0.083822   
2020-08-10      0.000781  0.016168  0.000061  0.037435  0.041911   
2020-08-11      0.024200  0.088084  0.100031  0.018717  0.145955   

Province_State                                                                \
Admin2               Bullock    Butler   Calhoun      Chambers      Cherokee   
2020-08-09      1.907356e-07  0.007145  0.046559  2.385008e-06  6.369706e-16   
2020-08-10      9.536782e-08  0.003572  0.130422  1.192504e-06  7.142857e-02   
2020-08-11      4.768391e-08  0.001786  0.094623  5.962520e-07  3.571429e-02   

Province_State  ... Wyoming                                                  \
Admin2          ...    Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-08-09      ...     NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-10      ...     NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-11      ...     NaN    NaN      NaN      NaN        NaN   NaN   NaN   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-08-09       0.005125      NaN    NaN  
2020-08-10       0.002563      NaN    NaN  
2020-08-11       0.019800      NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200811,AK,4589,286155,NaN,39.0,NaN,NaN,NaN,4.0,...,290744,0,0,75189f7e45df3d19bc61fc8790a6df68e33c41e4,0,0,0,0,0,NaN
1,20200811,AL,103851,679838,NaN,1506.0,12070.0,NaN,1270.0,NaN,...,783689,50,0,d775daf03cf949c25340db800bb25f4eec3c5fda,0,0,0,0,0,NaN
2,20200811,AR,50411,522457,NaN,507.0,3401.0,NaN,NaN,116.0,...,572868,11,65,484fcecae9e6dad1406aa0904f82d32c63d7d5b3,0,0,0,0,0,NaN
3,20200811,AS,0,1396,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,338e536f1145315e0475454c4e01ba26f5c4d156,0,0,0,0,0,NaN
4,20200811,AZ,188737,848538,NaN,1574.0,19292.0,510.0,NaN,346.0,...,1037275,45,15,86155d274c0b7a5e8590b83954b198d3f6deae91,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200811,AK,4589.0,286155.0,NaN,39.0,NaN,NaN,NaN,4.0,...,290744,0,0,75189f7e45df3d19bc61fc8790a6df68e33c41e4,0,0,0,0,0,NaN
1,20200811,AL,103851.0,679838.0,NaN,1506.0,12070.0,NaN,1270.0,NaN,...,783689,50,0,d775daf03cf949c25340db800bb25f4eec3c5fda,0,0,0,0,0,NaN
2,20200811,AR,50411.0,522457.0,NaN,507.0,3401.0,NaN,NaN,116.0,...,572868,11,65,484fcecae9e6dad1406aa0904f82d32c63d7d5b3,0,0,0,0,0,NaN
3,20200811,AS,0.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,338e536f1145315e0475454c4e01ba26f5c4d156,0,0,0,0,0,NaN
4,20200811,AZ,188737.0,848538.0,NaN,1574.0,19292.0,510.0,NaN,346.0,...,1037275,45,15,86155d274c0b7a5e8590b83954b198d3f6deae91,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-11,AK,4589,286155,NaN,39.0,NaN,NaN,NaN,4.0,NaN,...,290744,0,0,75189f7e45df3d19bc61fc8790a6df68e33c41e4,0,0,0,0,0,NaN
2020-08-11,AL,103851,679838,NaN,1506.0,12070.0,NaN,1270.0,NaN,691.0,...,783689,50,0,d775daf03cf949c25340db800bb25f4eec3c5fda,0,0,0,0,0,NaN
2020-08-11,AR,50411,522457,NaN,507.0,3401.0,NaN,NaN,116.0,450.0,...,572868,11,65,484fcecae9e6dad1406aa0904f82d32c63d7d5b3,0,0,0,0,0,NaN
2020-08-11,AS,0,1396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,338e536f1145315e0475454c4e01ba26f5c4d156,0,0,0,0,0,NaN
2020-08-11,AZ,188737,848538,NaN,1574.0,19292.0,510.0,NaN,346.0,NaN,...,1037275,45,15,86155d274c0b7a5e8590b83954b198d3f6deae91,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-11,AK,4589.0,286155.0,NaN,39.0,NaN,NaN,NaN,4.0,NaN,...,290744,0,0,75189f7e45df3d19bc61fc8790a6df68e33c41e4,0,0,0,0,0,NaN
2020-08-11,AL,103851.0,679838.0,NaN,1506.0,12070.0,NaN,1270.0,NaN,691.0,...,783689,50,0,d775daf03cf949c25340db800bb25f4eec3c5fda,0,0,0,0,0,NaN
2020-08-11,AR,50411.0,522457.0,NaN,507.0,3401.0,NaN,NaN,116.0,450.0,...,572868,11,65,484fcecae9e6dad1406aa0904f82d32c63d7d5b3,0,0,0,0,0,NaN
2020-08-11,AS,0.0,1396.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1396,0,0,338e536f1145315e0475454c4e01ba26f5c4d156,0,0,0,0,0,NaN
2020-08-11,AZ,188737.0,848538.0,NaN,1574.0,19292.0,510.0,NaN,346.0,NaN,...,1037275,45,15,86155d274c0b7a5e8590b83954b198d3f6deae91,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE